The aim of this investigation is to find the fraction of events that are observed within 24 hours.

libid_details['LIBID:', 'RA:', 'DECL:', 'NOBS:', 'MWEBV:', 'PIXSIZE:', 'first observation index', 'last observation index']

pointings = [LIBID, RA, DECL, MJD IDEXPT  FLT GAIN NOISE SKYSIG (pixels)  RATIO  ZPTAVG ZPTERR  MAG]

In [1]:
#import pointings 
from parse_simlib import parse_simlib

libid_details, pointings = parse_simlib('../minion_1016_WFD.simlib')

In [18]:
import numpy as np
import os

from python_utilities.des_io import parse_observations

def load_KN_locs(DIR): #load locations and times of all DAT files for KN in given directory 
    file_count = len([f for f in os.walk(DIR).next()[2] if f[-4:] == ".DAT"])
    KN_locations = np.zeros([file_count, 3])

    file_num = 0
    for file in os.listdir(DIR):  # run through all .dat files in the specified directory
        if file.endswith(".DAT"):
            obs, headerdict = parse_observations(DIR + file)
            KN_locations[file_num,:] = [headerdict['PEAKMJD'], headerdict['RA'], headerdict['DECL']]
            file_num += 1
    return KN_locations

KN_locations = load_KN_locs('../GW170817_AT2017gfo_LSST_WFD/')
            
print KN_locations[0:5]

[[  6.17753400e+04   6.90160900e+01  -5.85892330e+01]
 [  6.10670430e+04   1.16863968e+02  -5.08599170e+01]
 [  6.24793240e+04   1.57593414e+02  -4.21845890e+01]
 [  6.22443280e+04   1.87015884e+02  -3.69101140e+01]
 [  5.98204410e+04   3.16984100e+02  -3.69100000e+01]]


In [15]:
from astropy import units as u
from astropy.coordinates import SkyCoord
import scipy

#for each mjd find all pointings in the next 24 hours and find min separation, don't care about the band
def min_separation_24hours(event):
    #get pointings in the next 24 hours
    pointings_later = pointings[event[0]<pointings[:,3], :]
    pointings_next24 = pointings_later[pointings_later[:,3]<(event[0]+1)]
    
    #get angular separation of the pointings to the KN
    kilonova_loc = SkyCoord(ra=event[1] * u.degree, dec=event[2] * u.degree)
    pointings_locs = SkyCoord(ra=pointings_next24[:,1] * u.degree, dec=pointings_next24[:,2] * u.degree)
    angsep = kilonova_loc.separation(pointings_locs)
    
    #get min separation and the band of that observation
    if len(angsep.deg) !=0:
        min_sep = np.min(angsep.deg)
        band = pointings_next24[np.argmin(angsep.deg), 5]
    else: 
        min_sep = np.nan
        band = np.nan
    return min_sep, band

min_separation_24hours(KN_locations[0,:])

(10.930732422071442, 3.0)

In [19]:
#get metric for fraction of KN observed in next 24 hours
count_observed = 0
for num, KNlocation in enumerate(KN_locations):
    min_sep, band = min_separation_24hours(KNlocation)

    if min_sep != np.nan:
        if min_sep < np.sqrt(3):
            count_observed +=1 

fraction_observed = float(count_observed)/len(KN_locations)
print fraction_observed

0.593103448276


0.744827586207
